In [150]:
from itertools import product
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import (MaxAbsScaler, MinMaxScaler,
                                   PolynomialFeatures, RobustScaler,
                                   StandardScaler)
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import (BaseCrossValidator, GridSearchCV, KFold,
                                     RandomizedSearchCV, StratifiedKFold,
                                     check_cv, train_test_split)
from sklearn.linear_model import (ARDRegression, BayesianRidge, ElasticNet,
                                  ElasticNetCV, Lars, Lasso, LassoLars,
                                  LinearRegression, LogisticRegression,
                                  LogisticRegressionCV,
                                  OrthogonalMatchingPursuit, Ridge)
from sklearn.ensemble import (GradientBoostingClassifier,
                              GradientBoostingRegressor,
                              RandomForestClassifier, RandomForestRegressor)
from sklearn.base import BaseEstimator, is_regressor
import econml
from econml.orf import DMLOrthoForest
# from econml.metalearners import SLearner, TLearner, XLearner
from econml.grf import CausalForest
from econml.dr import DRLearner
from econml.dml import CausalForestDML, KernelDML, LinearDML, SparseLinearDML
import sklearn.preprocessing
import sklearn.neural_network
import sklearn.linear_model
import sklearn.ensemble
import sklearn
import argparse
import logging
import os
import pdb
import numpy as np
import pandas as pd
import pickle
import utils
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import time

During Cross-fitting

In [151]:
#Divide into k folds.
#For each fold, select a different model and find the best hyperparameter 
# get score, mse, time of estimators. Set cv=0 for estimator
def get_models_during_k_folds(X, T, Y, ci_estimator_list, model_y, model_t):
    k = 4
    cv = KFold(n_splits=k, shuffle=True, random_state=123)

    fold_models = {}
    i = 0
    for train_index, test_index in cv.split(X):
        X_train, T_train, Y_train = X.iloc[train_index], T.iloc[train_index], Y.iloc[train_index]
        X_test, T_test, Y_test = X.iloc[test_index], T.iloc[test_index], Y.iloc[test_index]
        # current_model_y = utils.select_continuous_estimator(model_y[i])
        # current_model_t = utils.select_discrete_estimator(model_t[i])
        current_model_y, current_model_t = model_y[i], model_t[i]
        print(current_model_y, type(current_model_y))
        causal_model_score = {}
        causal_model_mse = {}
        causal_model_time = {}
        for ci in ci_estimator_list:
            #find scores + mse's of all estimators
            causal_model = utils.get_estimators(ci, current_model_y, current_model_t)
            print(causal_model)
            start_time = time.time()
            causal_model.fit(Y_train, T_train, X=X_train, W=None)
            run_time = time.time() - start_time
            te_pred = causal_model.effect(X_test)
            causal_model_score[ci] = causal_model.score(Y_test, T_test, X_test)
            causal_model_mse[ci] = np.mean((Y_test - te_pred)**2)
            causal_model_time[ci] = run_time
        fold_models[f'fold {i}'] = {'model_y' : current_model_y, 'model_t' : current_model_t, 'Scores' : causal_model_score, 'Mse' : causal_model_mse, 'Runtime' : causal_model_time}
        i += 1

    return fold_models

In [140]:
data, X, T, Y, true_ITE, true_ATE, true_ATE_stderr, is_discrete = utils.load_ihdp()
# X, X_val, T, T_val, Y, Y_val = train_test_split(X, T, Y, train_size=0.6,shuffle=True, random_state=123)
# X_val, X_test, T_val, T_test, Y_val, Y_test = train_test_split(X_val, T_val, Y_val, train_size=.5, shuffle=True, random_state=123)

In [152]:
ci_estimator_list = ['dml']
model_y = ['linear', 'forest', 'gbf', 'nnet']
model_t = ['linear', 'forest', 'gbf', 'nnet']
# model_y = [ElasticNetCV(alphas=1), RandomForestRegressor(), GradientBoostingRegressor(), MLPRegressor()]
# model_t = [LogisticRegressionCV(max_iter=500), RandomForestClassifier(), GradientBoostingClassifier(), MLPClassifier()]

In [153]:
fold_models = get_models_during_k_folds(X, T, Y, ci_estimator_list, model_y, model_t)
fold_models

linear <class 'str'>


No scoring value was given. Using default score method neg_mean_squared_error.
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.172e-01, tolerance: 1.036e-01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.615e-01, tolerance: 1.036e-01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.624e-01, tolerance: 1.036e-01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.435e-01, tolerance: 1.036e-01
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

forest <class 'str'>
gbf <class 'str'>


No scoring value was given. Using default score method neg_mean_squared_error.


nnet <class 'str'>


No scoring value was given. Using default score method neg_mean_squared_error.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.


{'fold 0': {'model_y': 'linear',
  'model_t': 'linear',
  'Scores': {'dml': 1.2595245103007908},
  'Mse': {'dml': 6.577383518859473},
  'Runtime': {'dml': 110.76304578781128}},
 'fold 1': {'model_y': 'forest',
  'model_t': 'forest',
  'Scores': {'dml': 2.2472905164183956},
  'Mse': {'dml': 10.968860822579092},
  'Runtime': {'dml': 9.771544933319092}},
 'fold 2': {'model_y': 'gbf',
  'model_t': 'gbf',
  'Scores': {'dml': 2.318973169901894},
  'Mse': {'dml': 8.352729413896165},
  'Runtime': {'dml': 3.3521230220794678}},
 'fold 3': {'model_y': 'nnet',
  'model_t': 'nnet',
  'Scores': {'dml': 10.780917041541795},
  'Mse': {'dml': 6.7527878942980974},
  'Runtime': {'dml': 5.572843313217163}}}